#¿Qué es esto?
El valor de la auditoría SEO está en el análisis y la interpretación de los datos, no en la exportación, limpieza ni unificación de datos. Por esto se creo este script que automatiza la unificación de los archivos de Screaming frog. dejando más tiempo para la interpretación de datos.



**¿qué evalua esta auditoría?**

#Rastreo

* Si hay errores 4XX.
* Si hay respuestas 3XX.


#Cor Web vitals
* Hay urls con elementos potencialmente bloqueantes.
* Hay urls con elementos con imagenes en CSS por minificar.
* Hay urls con elementos con imagenes en JS por minificar.
* Si hay imágenes sobre escaladas.
* Si hay imágenes con formatos de última generación implementadas.
* SI hay imagenes que necesite aplicar lazy loading.


#On-Page
##H1

1. Si hay páginas sin h1.
2. Si hay páginas con h1 duplicado.
3. Si hay h1 con más de 70 caracteres.
4. Si hay h1 multiples.

##Meta titulos

5. Si hay páginas sin titles.
6. Si hay titulos duplicados.
7. Si hay páginas con titulos superiores a los 60 carácteres.
8. Si hay páginas con titulos inferiores a los 30 caractetes.

##Meta descripción

10. Páginas que no tengan meta descripción.
11. Páginas con meta descripción duplicada.
12. Páginas con meta descripción superior a los 155 caracteres.
13. páginas con meta descripción inferior a los 70 caracteres.
14. páginas con multiples meta descripciones.

##Imágenes
15. Imágenes que superen los 180 KB.
16. Imágenes que no tengan texto alternativo.
17. Imágenes sin atributo Alt-text

#Popularidad
18. Número de backlinks.
19. Cantidad de textos ancla.

##inicilizar variables e importar librerías

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import date

from google.colab import drive
drive.mount('/content/drive')

#Día actual
today = date.today()
site_name = "https://cendiatra.com/"

#explanation about Rastreo
whatis_http = "Si algún enlace del sitio web señala a una versión HTTP antigua del mismo, los motores de búsqueda pueden confundirse sobre cuál versión de la página incluir en el ranking."
whatis_canonocalizacion = "Al configurar un elemento rel='canonical' en tu página, puedes informar a los motores de búsqueda cuál versión de la página deseas que aparezca en los resultados de búsqueda. Al utilizar etiquetas canónicas, es importante asegurarse de que la URL que incluyes en tu elemento rel='canonical' dirija a una página que realmente existe. Los enlaces canónicos que dirigen a páginas web inexistentes complican el proceso de rastreo e indexación del contenido y, como resultado, reducen la eficiencia del rastreo y desperdician innecesariamente tu presupuesto de rastreo."
whatis_indexing_directives = "X-robots-tag es un encabezado HTTP que puede utilizarse para indicar a los motores de búsqueda si indexar o rastrear la página web o no. Esta etiqueta admite las mismas directivas que una etiqueta meta robots común y se utiliza normalmente para controlar el rastreo de archivos distintos de HTML. Si el rastreo de una página se bloquea mediante x-robots-tag, esta nunca aparecerá en los resultados de búsqueda."
whatis_hreflang = "Un atributo hreflang (rel='alternate' hreflang='x') ayuda a los motores de búsqueda a comprender qué página debe mostrarse a los visitantes en función de su ubicación. Es necesario usar este atributo si tu sitio web es multilingüe y deseas ayudar a los usuarios de otros países a encontrar su contenido en el idioma más apropiado para ellos."

#explanation about indexing
whatis_4xx = "Un error 4xx significa que no se puede acceder a una página web. Esto suele ser el resultado de enlaces rotos. Estos errores impiden que los usuarios y robots de los motores de búsqueda accedan a tus páginas web y pueden influir negativamente tanto en la experiencia de usuario como en la rastreabilidad de los motores de búsqueda. Esto, a su vez, conlleva un descenso del tráfico dirigido a tu sitio web"
whatis_3xx = "Redirigir una URL a otra es adecuado en muchas situaciones. Sin embargo, si los redireccionamientos se realizan de manera incorrecta, pueden producir resultados desastrosos. Dos ejemplos comunes del uso indebido del redireccionamiento son las cadenas y bucles de redireccionamiento"


#explanation about core web vitals
whatis_Remove_Render_Blocking_Resources = "A medida que se analiza el HTML, cualquier hoja de estilo o CSS que se encuentre hará que el analizador se detenga, lo que retrasará el LCP. Idealmente, aplazaría cualquier CSS y JavaScript no críticos para acelerar la carga de su contenido principal."
whatis_Minify_CSS = "Minimización es la práctica de eliminar el código de caracteres innecesarios para reducir su tamaño, mejorando así los tiempos de carga. Cuando el código es minimizado todos los comentarios se eliminan, así como los espacios en blanco y caracteres innecesarios (espacio, nueva línea…)."
whatis_Minify_JavaScript ="Minimización es la práctica de eliminar el código de caracteres innecesarios para reducir su tamaño, mejorando así los tiempos de carga. Cuando el código es minimizado todos los comentarios se eliminan, así como los espacios en blanco y caracteres innecesarios (espacio, nueva línea…)."
whatis_Properly_Size_Images = "Su página nunca debe mostrar imágenes que sean más grandes que la versión que se muestra en la pantalla del usuario. Cualquier cosa más grande que eso solo da como resultado bytes desperdiciados y ralentiza el tiempo de carga de la página."
whatis_lazy_loading ="Se llama carga diferida al enfoque de esperar para cargar los recursos hasta que se necesiten, en lugar de cargarlos por adelantado. Esto puede mejorar el rendimiento al reducir la cantidad de recursos que deben cargarse y analizarse en la carga inicial de la página."
whatis_webp ="AVIF y WebP son formatos de imagen que tienen características superiores de compresión y calidad en comparación con sus equivalentes JPEG y PNG más antiguos. Codificar sus imágenes en estos formatos en lugar de JPEG o PNG significa que se cargarán más rápido y consumirán menos datos móviles."

#explanation about h1
whatis_h1_less = "Aunque menos importantes que las etiquetas <title>, los encabezados h1 te ayudarán a definir el tema de tu página para los motores de búsqueda y usuarios. Si falta una etiqueta <h1> o está vacía, los motores de búsqueda pueden colocar tu página en un ranking inferior del que tendría. Además, si falta una etiqueta <h1> se rompe la jerarquía de encabezados de tu página, lo cual influye negativamente en la SEO"
whatis_h1_duplicate = "Es mala idea duplicar el contenido de la etiqueta del título en tu encabezado de primer nivel. Si las etiquetas <title> y <h1> de tu página coinciden, estas últimas puede que aparezcan demasiado optimizadas para los motores de búsqueda.Además, si se utiliza el mismo contenido en títulos y encabezados se pierde la oportunidad de incorporar otras palabras clave relevantes para tu página."
whatis_h1_multiple = "Es mala idea tener mùltiples etiquetas del título en tu encabezado de primer nivel. Si las etiqueta <h1> de tu página aparece varias veces, confunden a los rastreadores y por ende a tus usuarios ya que se identifican como títulos del contenido de tu página."
whatis_h1_more_70 = "es una mala idea tener h1 superiores a los 70 caracteres, ya que Google puede escoger no mostrar la etiqeuta <title> y poner en su lugar el h1 por esta razón los h1 debe´rian estar optimizados en su extensión como un title."

#explanation about title
whatis_less_title = "Una etiqueta <title> es un elemento SEO integrado en la página. Aparece en navegadores y resultados de búsqueda y ayuda tanto a los motores de búsqueda como a los usuarios a comprender de qué se trata tu página."
whatis_duplicate_title ="Una etiqueta <title> es un elemento SEO integrado en la página. Aparece en navegadores y resultados de búsqueda y ayuda tanto a los motores de búsqueda como a los usuarios a comprender de qué se trata tu página. Es importante solo tengas una por página"
whatis_more_60_title = "La mayoría de motores de búsqueda recortan títulos que incluyan más de 70 caracteres. Los títulos cortos e incompletos tienen un aspecto poco atractivo para los usuarios y no les animarán a hacer clic en tu página."
whatis_less_30_title = "Una etiqueta <title> es un elemento SEO integrado en la página. Aparece en navegadores y resultados de búsqueda y ayuda tanto a los motores de búsqueda como a los usuarios a comprender de qué se trata tu página.Si a una página tiene un meta título inferior a los 30 caracteres, podría estar perdiendo oportunidades para agregar más palabras relacionadas ."

#explanation about metadescription
whatis_less_description ="A pesar de que las metadescripciones no influyen directamente en los rankings, los motores de búsqueda las utilizan para mostrar la descripción de tu página en los resultados de búsqueda. Una buena descripción ayuda a los usuarios a saber de qué trata tu página y les anima a hacer clic en ella. Si falta la etiqueta de metadescripción de tu página, los motores de búsqueda generalmente mostrarán su primera oración, que puede ser irrelevante y poco atractiva para los usuarios."
whatis_duplicated_description = "Una etiqueta <meta description> es un breve resumen del contenido de una página web que ayuda a los motores de búsqueda a comprender de qué se trata tu página y puede mostrarse a los usuarios en los resultados de búsqueda.Las metadescripciones duplicadas en diferentes páginas constituyen una oportunidad perdida de utilizar palabras clave más relevantes. Además, las metadescripciones duplicadas dificultan a los motores de búsqueda y a los usuarios diferenciar entre las diferentes páginas web. Es mejor no tener ninguna metadescripción que tener una duplicada."
whatis_more_155_description = "A pesar de que las metadescripciones no influyen directamente en los rankings, los motores de búsqueda las utilizan para mostrar la descripción de tu página en los resultados de búsqueda. Una buena descripción ayuda a los usuarios a saber de qué trata tu página y les anima a hacer clic en ella. Si la página tiene una meta descripción demasiado larga no se mostrará completa en los SERPs"

#explanation about images
whatis_img_more_180kb = "La velocidad de carga de una página es uno de los factores más importantes para el ranking. Cuanto menos tarde la página en cargar, más alto es el ranking que puede recibir. Además, las páginas que se cargan rápido influyen positivamente en la experiencia de usuario y pueden aumentar tus tasas de conversión. una página con un imágenes superiores a los 180 kb puede afectar drásticamente la carga general del sitio sin embargo en algunos casos se puede hacer exepciones."
whatis_images_alt_less = "Los motores de búsqueda utilizan los atributos alt en las etiquetas <img> para comprender el contenido de tus imágenes. Si descuidas los atributos alt, puedes perder la oportunidad de obtener un mejor posicionamiento en los resultados de búsqueda porque ellos te permiten ocupar un mejor ranking en los resultados de búsquedas de imágenes. Si no utilizas atributos alt, esto influye negativamente en la experiencia de usuarios con discapacidades visuales y de aquellos que han desactivado las imágenes en sus navegadores."
whatis_missing_alt_attribute = "Los motores de búsqueda utilizan los atributos alt en las etiquetas <img> para comprender el contenido de tus imágenes. Si tu sitio no tiene implementados los atributos alt en las imágenes, pierdes la oportunidad de obtener un mejor posicionamiento en los resultados de búsqueda porque ellos te permiten ocupar un mejor ranking en los resultados de búsquedas de imágenes. Si no utilizas atributos alt, esto influye negativamente en la experiencia de usuarios con discapacidades visuales y de aquellos que han desactivado las imágenes en sus navegadores."

#explanation about popularity
whatis_autority_score ="El Authority Score es una métrica compuesta, diseñada por SEMrush, que mide la calidad general y el rendimiento SEO de un dominio o la página de una web. Esta puntuación se basa en muchas métricas (más detalles a continuación) que representa confianza y autoridad. Asimismo, utiliza una red neuronal artificial y el aprendizaje automático para asegurarse de que la puntuación esté en todo momento actualizada y relevante."
whatis_backlinkscount ="Un backlink es sencillamente un enlace desde otro sitio web que apunta al nuestro. Cuantos más backlinks de calidad consigas generar, mejor posicionado estará tu sitio web."
whatis_anchortext ="El texto de anclaje o texto ancla es el texto o frase que muestras en los enlaces que incluyes en tus contenidos web. Se utiliza el término “anclaje” porque sirve de conexión entre dos ubicaciones o recursos en Internet."

#Inicializar variables
df_404= pd.DataFrame
df_301= pd.DataFrame
df_Remove_Render_Blocking_Resources = pd.DataFrame
df_Minify_CSS = pd.DataFrame
df_Minify_JavaScript = pd.DataFrame
df_Properly_Size_Images= pd.DataFrame
df_lazyloading= pd.DataFrame
df_webp= pd.DataFrame

df_falta_h1= pd.DataFrame
df_duplicated_h1= pd.DataFrame
df_more_70_h1= pd.DataFrame
df_multiple_h1= pd.DataFrame
df_title_less= pd.DataFrame
df_duplicated_titles= pd.DataFrame
df_titles_more_60= pd.DataFrame
df_titles_less_30= pd.DataFrame
df_description_less= pd.DataFrame
df_duplicate_description= pd.DataFrame
df_description_more_155= pd.DataFrame
df_description_less_70= pd.DataFrame
df_multiple_description= pd.DataFrame
df_img_more_180kb= pd.DataFrame
df_alt_less= pd.DataFrame
df_missing_alt_attribute= pd.DataFrame
df_domain_count = pd.DataFrame
df_anchortext = pd.DataFrame



#compilación de conteo
count_404 = 0
count_301 = 0
text_estimation_404 = 0
text_estimation_301 = 0
count_Remove_Render_Blocking = 0
text_estimation_Remove_Blocking = 0
count_Minify_CSS = 0
text_Minify_CSS = 0
count_Minify_JavaScript= 0
text_Minify_JavaScript = 0
count_lazy = 0
text_lazyloading = 0
text_webp = 0
count_webp = 0
text_Properly_Size = 0
count_Properly_Size_Images = 0
count_falta_h1 = 0
text_falta_h1 = 0
count_duplicated_h1 = 0
text_duplicated_h1 = 0
count_more_70_h1 = 0
text_more_70_h1 = 0
count_multiple_h1 = 0
count_titles_more_60 = 0
text_titles_more_60 = 0
count_titles_less_30 = 0
text_titles_less_30 = 0
text_titles_less = 0
count_titles_less = 0
count_duplicate_title = 0
text_duplicate_title = 0
count_description_less  = 0
text_description_less = 0
count_duplicate_description = 0
text_duplicate_description = 0
count_description_more_155 = 0
text_description_more_155 = 0
count_alt_less = 0
text_alt_less = 0
count_img_more_180kb = 0
text_img_more_180kb = 0
count_missing_alt_attribute = 0
text_missing_alt_attribute = 0
count_domains = 0
text_domains = 0
county_anchortext = 0
text_anchortext = 0

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Importación de datos

In [ ]:
#indexabilidad

#páginas con códigos de respuesta 4xx
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/códigos_de_respuesta_error_de_cliente_(4xx).csv') == True:
  df_404 = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/códigos_de_respuesta_error_de_cliente_(4xx).csv")
else:
  pass

#páginas con códigos de redirección 301
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/códigos_de_respuesta_redirección_(3xx).csv') == True:
  df_301 =pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/códigos_de_respuesta_redirección_(3xx).csv")
else:
  pass

#Cor web vitals

#bloking rendering pages
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/pagespeed_eliminar_recursos_de_bloqueo_de_renderizado.csv') == True:
  df_Remove_Render_Blocking_Resources = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/pagespeed_eliminar_recursos_de_bloqueo_de_renderizado.csv")
else:
  pass

# Minificar CSS
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/pagespeed_minimizar_css.csv') == True:
  df_Minify_CSS = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/pagespeed_minimizar_css.csv")
else:
  pass

#Minificar JS
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/pagespeed_minimizar_javascript.csv') == True:
  df_Minify_JavaScript = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/pagespeed_minimizar_javascript.csv")
else:
  pass

#Scale size images
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/pagespeed_dimensionar_imágenes_correctamente.csv') == True:
  df_Properly_Size_Images = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/pagespeed_dimensionar_imágenes_correctamente.csv")
else:
  pass

#Imágenes que tengan carga diferida implementada.
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/pagespeed_aplazar_imágenes_fuera_de_pantalla.csv') == True:
  df_lazyloading = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/pagespeed_aplazar_imágenes_fuera_de_pantalla.csv")
else:
  pass

#Mostrar imágenes en formato de última generación.
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/pagespeed_mostrar_imágenes_en_formatos_de_última_generación.csv') == True:
  df_webp = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/pagespeed_mostrar_imágenes_en_formatos_de_última_generación.csv")
else:
  pass


#Páginas sin h1.
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/h1_falta.csv') == True:
  df_falta_h1 = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/informes/h1_falta.csv")
else:
  pass

#h1 duplicado.
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/h1_duplicado.csv') == True:
  df_duplicated_h1 = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/h1_duplicado.csv")
else:
  pass

#más de 70 caracteres.
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/h1_más_de_70 caracteres.csv') == True:
  df_more_70_h1 = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/h1_más_de_70 caracteres.csv")
else:
  pass

#h1 multiples.
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/h1_múltiple.csv') == True:
  df_multiple_h1 = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/h1_múltiple.csv")
else:
  pass

#páginas sin titles
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/títulos_de_página_falta.csv') == True:
  df_title_less = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/títulos_de_página_falta.csv")
else:
  pass

#titulos duplicados
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/títulos_de_página_duplicado.csv') == True:
  df_duplicated_titles = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/títulos_de_página_duplicado.csv")
else:
  pass

#titulos superiores a los 60 carácteres.
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/títulos_de_página_más_de_60 caracteres.csv') == True:
  df_titles_more_60 = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/títulos_de_página_más_de_60 caracteres.csv")
else:
  pass

#titulos inferiores a los 30 caractetes.
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/títulos_de_página_menos_de_30 caracteres.csv') == True:
  df_titles_less_30 = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/títulos_de_página_menos_de_30 caracteres.csv")
else:
  pass

#meta descripción

#sin meta descripción
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/meta_description_falta.csv') == True:
  df_description_less = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/meta_description_falta.csv")
else:
  pass

#con meta descripción duplicada
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/meta_description_duplicado.csv') == True:
  df_duplicate_description = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/meta_description_duplicado.csv")
else:
  pass

#con meta descripción superior a los 155 caracteres.
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/meta_description_más_de_155 caracteres.csv') == True:
  df_description_more_155 = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/meta_description_más_de_155 caracteres.csv")
else:
  pass

#con meta descripción inferior a los 70 caracteres.
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/meta_description_menos_de_70 caracteres.csv') == True:
  df_description_less_70 = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/meta_description_menos_de_70 caracteres.csv")
else:
  pass

#con multiples meta descripciones
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/meta_description_múltiple.csv') == True:
  df_multiple_description = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/meta_description_múltiple.csv")
else:
  pass

#images

#imagenes con más de 180kb de peso.
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/imágenes_más_de_180 kb.csv') == True:
  df_img_more_180kb = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/imágenes_más_de_180 kb.csv")
else:
  pass

#imagenes sin textos alternativos.
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/imágenes_falta_texto_alt.csv') == True:
  df_alt_less = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/imágenes_falta_texto_alt.csv")
else:
  pass

#imagenes sin atributo texto alternativo.
if os.path.exists('/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/imágenes_falta_atributo_alt.csv') == True:
  df_missing_alt_attribute = pd.read_csv("/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Informes/imágenes_falta_atributo_alt.csv")
else:
  pass


##Limpieza de datos


In [ ]:
#indexabilidad
if df_404.empty:
    pass
else:
  df_404_clear= df_404.loc[:, ['Dirección','Código de respuesta','Respuesta']]
  df_404_clear['Fecha'] = today
  count_404 = df_404_clear['Dirección'].size
  time_404 = 3
  estimation_404 = count_404 *time_404 /60
  text_estimation_404 = 'Se requieren próximadamente '+ str(estimation_404) +' horas'

if df_301.empty:
    pass
else:
  df_301_clear= df_301.loc[:, ['Dirección']]
  df_301_clear['Fecha'] = today
  count_301 = df_301_clear['Dirección'].size
  time_301 = 3
  estimation_301 = count_301 *time_301 /60
  text_estimation_301 = 'Se requieren apróximadamente '+ str(estimation_301) +' horas'

#Cor web vitals

if df_Remove_Render_Blocking_Resources.empty:
    pass
else:
  df_Remove_Render_Blocking_Resources_clear= df_Remove_Render_Blocking_Resources.loc[:, ['Dirección','Ahorro al eliminar los recursos que bloquean el renderizado (ms)']]
  df_Remove_Render_Blocking_Resources_clear['Fecha'] = today
  count_Remove_Render_Blocking = df_Remove_Render_Blocking_Resources_clear['Dirección'].size
  time_Remove_Render_Blocking = 10
  estimation_Remove_Blocking = count_Remove_Render_Blocking * time_Remove_Render_Blocking /60
  text_estimation_Remove_Blocking = 'Se requieren apróximadamente '+ str(estimation_Remove_Blocking) +' horas'

if df_Minify_CSS.empty:
    pass
else:
  df_Minify_CSS_clear= df_Minify_CSS.loc[:, ['Dirección','Ahorro al minimizar CSS (ms)']]
  df_Minify_CSS_clear['Fecha'] = today
  count_Minify_CSS = df_Minify_CSS_clear['Dirección'].size
  time_Minify_CSS = 10
  estimation_Minify_CSS = count_Minify_CSS * time_Minify_CSS /60
  text_estimation_Minify_CSS = 'Se arequieren próximadamente '+ str(estimation_Minify_CSS) +' horas'

if df_Minify_JavaScript.empty:
    pass
else:
  df_Minify_JavaScript_clear= df_Minify_JavaScript.loc[:, ['Dirección']]
  df_Minify_JavaScript_clear['Fecha'] = today
  count_Minify_JavaScript = df_Minify_JavaScript_clear['Dirección'].size
  time_Minify_JavaScript = 10
  estimation_Minify_JavaScript = count_Minify_JavaScript * time_Minify_JavaScript /60
  text_Minify_JavaScript = 'Se requieren apróximadamente '+ str(estimation_Minify_JavaScript) +' horas'

if df_Properly_Size_Images.empty:
    pass
else:
  df_Properly_Size_Images_clear = df_Properly_Size_Images.loc[:, ['Dirección','Ahorro por tamaño adecuado de las imágenes (ms)']]
  df_Properly_Size_Images_clear['Fecha'] = today
  count_Properly_Size_Images = df_Properly_Size_Images_clear['Dirección'].size
  time_Properly_Size = 5
  estimation_Properly_Size = count_Properly_Size_Images * time_Properly_Size /60
  text_Properly_Size = 'Se requieren apróximadamente '+ str(estimation_Properly_Size) +' horas'

if df_lazyloading.empty:
    pass
else:
  df_lazyloading_clear = df_lazyloading.loc[:, ['Dirección','Ahorro al posponer las imágenes fuera de la pantalla (ms)']]
  df_lazyloading_clear['Fecha'] = today
  count_lazy = df_lazyloading_clear['Dirección'].size
  time_lazyloading = 5
  estimation_lazyloading = count_lazy * time_lazyloading /60
  text_lazyloading = 'Se requieren apróximadamente '+ str(estimation_lazyloading) +' horas'


if df_webp.empty:
    pass
else:
  df_webp_clear = df_webp.loc[:, ['Dirección','Ahorro al mostrar imágenes en formato de última generación (ms)']]
  df_webp_clear['Fecha'] = today
  count_webp = df_webp_clear['Dirección'].size
  time_webp = 7
  estimation_webp = count_webp * time_webp /60
  text_webp = 'Se requieren apróximadamente '+ str(estimation_lazyloading) +' horas'

#H1

if df_falta_h1.empty:
    pass
else:
  df_falta_h1_clear= df_falta_h1.loc[:, ['Dirección','Indexabilidad']]
  df_falta_h1_clear['Fecha'] = today
  count_falta_h1 = df_falta_h1_clear['Dirección'].size
  time_falta_h1 = 10
  estimation_falta_h1 = count_falta_h1 * time_falta_h1 /60
  text_falta_h1 = 'Se requieren apróximadamente '+ str(estimation_falta_h1) +' horas'

if df_duplicated_h1.empty:
    pass
else:
  df_duplicated_h1_clear= df_duplicated_h1.loc[:, ['Dirección','Repeticiones','H1-1','Longitud de H1-1']]
  df_duplicated_h1_clear['Fecha'] = today
  count_duplicated_h1 = df_duplicated_h1_clear['Dirección'].size
  time_duplicated_h1 = 10
  estimation_duplicated_h1 = count_duplicated_h1 * time_duplicated_h1 /60
  text_duplicated_h1 = 'Se requieren apróximadamente '+ str(estimation_duplicated_h1) +' horas'

if df_duplicated_h1.empty:
    pass
else:
  df_duplicated_h1_clear= df_duplicated_h1.loc[:, ['Dirección','Repeticiones','H1-1','Longitud de H1-1']]
  df_duplicated_h1_clear['Fecha'] = today
  count_duplicated_h1 = df_duplicated_h1_clear['Dirección'].size
  time_duplicated_h1 = 10
  estimation_duplicated_h1 = count_duplicated_h1 * time_duplicated_h1 /60
  text_duplicated_h1 = 'Se requieren apróximadamente '+ str(estimation_duplicated_h1) +' horas'

if df_more_70_h1.empty:
    pass
else:
  df_more_70_h1_clear= df_more_70_h1.loc[:, ['Dirección','Repeticiones','H1-1','Longitud de H1-1']]
  df_more_70_h1_clear['Fecha'] = today
  count_more_70_h1 = df_more_70_h1_clear['Dirección'].size
  time_more_70_h1 = 10
  estimation_more_70_h1 = count_more_70_h1 * time_more_70_h1 /60
  text_more_70_h1 = 'Se requieren apróximadamente '+ str(estimation_more_70_h1) +' horas'

#title
if df_titles_more_60.empty:
    pass
else:
  df_titles_more_60_clear= df_titles_more_60.loc[:, ['Dirección','Repeticiones','Título 1','Longitud del título 1']]
  df_titles_more_60_clear['Fecha'] = today
  count_titles_more_60 = df_titles_more_60_clear['Dirección'].size
  time_titles_more_60 = 10
  estimation_titles_more_60 = count_titles_more_60 * time_titles_more_60 /60
  text_titles_more_60 = 'Se requieren apróximadamente '+ str(estimation_titles_more_60) +' horas'

if df_titles_less_30.empty:
    pass
else:
  df_titles_less_30_clear= df_titles_less_30.loc[:, ['Dirección','Repeticiones','Título 1','Longitud del título 1']]
  df_titles_less_30_clear['Fecha'] = today
  count_titles_less_30 = df_titles_less_30_clear['Dirección'].size
  time_titles_less_30 = 10
  estimation_titles_less_30 = count_titles_less_30 * time_titles_less_30 /60
  text_titles_less_30 = 'Se requieren apróximadamente '+ str(estimation_titles_less_30) +' horas'

if df_title_less.empty:
    pass
else:
  df_title_less_clear= df_title_less.loc[:, ['Dirección','Indexabilidad']]
  df_title_less_clear['Fecha'] = today
  count_titles_less = df_title_less_clear['Dirección'].size
  time_titles_less = 10
  estimation_titles_less = count_titles_less * time_titles_less /60
  text_titles_less = 'Se requieren apróximadamente '+ str(estimation_titles_less) +' horas'

if df_duplicated_titles.empty:
    pass
else:
  df_duplicated_titles_clear= df_duplicated_titles.loc[:, ['Dirección','Indexabilidad']]
  df_duplicated_titles_clear['Fecha'] = today
  count_duplicate_title = df_duplicated_titles_clear['Dirección'].size
  time_duplicate_title = 10
  estimation_duplicate_title = count_duplicate_title * time_duplicate_title /60
  text_duplicate_title = 'Se requieren apróximadamente '+ str(estimation_duplicate_title) +' horas'

  #description

if df_description_less.empty:
    pass
else:
  df_description_less_clear= df_description_less.loc[:, ['Dirección','Repeticiones','Indexabilidad']]
  df_description_less_clear['Fecha'] = today
  count_description_less = df_description_less_clear['Dirección'].size
  time_description_less = 10
  estimation_description_less = count_description_less* time_description_less /60
  text_description_less = 'Se requieren apróximadamente '+ str(estimation_description_less) +' horas'

if df_duplicate_description.empty:
    pass
else:
  df_duplicate_description_clear= df_duplicate_description.loc[:, ['Dirección','Repeticiones','Meta description 1','Longitud de la meta description 1']]
  df_duplicate_description_clear['Fecha'] = today
  count_duplicate_description = df_duplicate_description_clear['Dirección'].size
  time_duplicate_description = 10
  estimation_duplicate_description = count_duplicate_description * time_duplicate_description /60
  text_duplicate_description = 'Se requieren apróximadamente '+ str(estimation_duplicate_description) +' horas'

if df_description_more_155.empty:
    pass
else:
  df_description_more_155_clear= df_description_more_155.loc[:, ['Dirección','Meta description 1','Longitud de la meta description 1','Longitud de la meta description 1']]
  df_description_more_155_clear['Fecha'] = today
  count_description_more_155= df_description_more_155_clear['Dirección'].size
  time_description_more_155 = 10
  estimation_description_more_155 = count_description_more_155 * time_description_more_155 /60
  text_description_more_155 = 'Se requieren apróximadamente '+ str(estimation_description_more_155) +' horas'

#images
if df_alt_less.empty:
    pass
else:
  df_alt_less_clear = df_alt_less.loc[:, ['Dirección','Tamaño (bytes)','Indexabilidad']]
  df_alt_less_clear['Fecha'] = today
  count_alt_less = df_alt_less_clear['Dirección'].size
  time_alt_less = 5
  estimation_alt_less = count_alt_less * time_alt_less /60
  text_alt_less = 'Se requieren apróximadamente '+ str(estimation_alt_less) +' horas'

if df_img_more_180kb.empty:
    pass
else:
  df_img_more_180kb_clear = df_img_more_180kb.loc[:, ['Dirección','Tamaño (bytes)','Indexabilidad']]
  df_img_more_180kb_clear['Fecha'] = today
  count_img_more_180kb = df_img_more_180kb_clear['Dirección'].size
  time_img_more_180kb = 7
  estimation_180kb = count_img_more_180kb * time_img_more_180kb /60
  text_180kb = 'Se requieren apróximadamente '+ str(estimation_180kb) +' horas'

if df_missing_alt_attribute.empty:
    pass
else:
  df_missing_alt_attribute_clear = df_missing_alt_attribute.loc[:, ['Dirección','Tamaño (bytes)','Indexabilidad']]
  df_missing_alt_attribute_clear['Fecha'] = today
  count_missing_alt_attribute = df_missing_alt_attribute_clear['Dirección'].size
  time_missing_alt_attribute = 10
  estimation_missing_alt_attribute = count_missing_alt_attribute * time_missing_alt_attribute /60
  text_missing_alt_attribute = 'Se requieren apróximadamente '+ str(estimation_missing_alt_attribute) +' horas'

#Autoridad
if df_domain_count.empty:
    pass
else:
  df_domain_count_clear = df_domain_count.loc[:, ['Page ascore','Source url','Target url']]
  df_domain_count_clear['Fecha'] = today
  count_domains = df_domain_count_clear['Source url'].size
  time_domains = 10
  estimation_domains = count_domains * time_domains /60
  text_domains = 'Se requieren apróximadamente '+ str(estimation_domains) +' horas'

if df_anchortext.empty:
    pass
else:
  df_anchortext_clear = df_anchortext.loc[:, ['Anchor','Domains','Backlinks']]
  df_anchortext_clear['Fecha'] = today
  county_anchortext = df_anchortext_clear['Domains'].size
  time_anchortext = 10
  estimation_anchortext = county_anchortext * time_anchortext /60
  text_anchortext = 'Se requieren apróximadamente '+ str(estimation_anchortext) +' horas'

expli_fecha ="La auditoría fue realizada con métricas e información de " + str(today)

df_matriz= pd.DataFrame({'Matriz de entendimiento': ["Esta auditoría SEO se realiza como punto de partida para la ejecución de acciones que mejoren el sitio" + site_name + ". Se hizo bajo la metología de RICOPA en la cual se prioriza desde el rastreo hasta el análisis final.",expli_fecha, "Objetivos: con esta actividad el estratega tendrá la capacidad de:1 Conocer las oportunidades en SEO e interiorizar la mejor forma de arreglarlo(Pestaña de resumen columna D).2. Priorizar las actividades para asignar según su prioridad que está clasificada en: Baja, media y alta(Pestaña de resumen columna F).3. Identificar el rol adecuado para realizar dicha implementación según sea el caso(Pestaña de resumen columna G)"]})
Resumen= {
  'RICOPA':['Rastreo',
            'Rastreo',

            'Rastreo',
            'Rastreo',
            'Rastreo',
            'Rastreo',
            'Rastreo',
            'Rastreo',

            'On-Page',
            'On-Page',
            'On-Page',

            'On-Page',
            'On-Page',
            'On-Page',
            'On-Page',

            'On-Page',
            'On-Page',
            'On-Page',

            'Contenido',
            'Contenido',
            'Contenido',

            'Popularidad',
            'Popularidad'],
  'FACTORES':['¿Hay URLs con respuesta 4XX?',
              '¿Hay redirecciones 3XX por auditar?',

              'Hay urls que tienen recursos potencialmente bloqueantes',
              'Hay url con hojas de Estilos(CSS) por minificar',
              'Hay urls con JavaScript (JS) por minificar',
              'Hay urls con imágenes sobre escaladas por optimizar',
              'Se puede implementar (WebP)',
              'Hay oportunidades de implementar cargadiferida',

              'URLs que no tienen H1',
              'URLs que tienen H1 duplicados',
              'URLs que tienen H1 con más de 70 caracteres',

              '¿Hay Páginas sin meta títulos',
              '¿Hay Páginas con meta títulos duplicados',
              '¿Hay meta títulos muy cortos? -30 Caracteres',
              '¿Hay meta títulos muy largos? +60 Caracteres',

              'Hay Páginas sin descripción',
              'Hay Páginas con descripción duplicada',
              'Hay Páginas con descripción superior a los 155 Caracteres',

              'Hay imágenes con peso superior a 180kB',
              'Hay imágenes sin texto alternativo',
              'Hay imágenes sin atributo de texto alternativo',

              '¿Cuáles son los backlinks de la compañía?',
              'Cuáles son los textos anclas más frecuentes'],
  'DESCRIPCIÓN DE FACTORES':[whatis_4xx,
                             whatis_3xx,

                             whatis_Remove_Render_Blocking_Resources,
                             whatis_Minify_CSS,
                             whatis_Minify_JavaScript,
                             whatis_Properly_Size_Images,
                             whatis_webp,
                             whatis_lazy_loading,

                             whatis_h1_less,
                             whatis_h1_duplicate,
                             whatis_h1_more_70,

                             whatis_less_title,
                             whatis_duplicate_title,
                             whatis_less_30_title,
                             whatis_more_60_title,

                             whatis_less_description,
                             whatis_duplicated_description,
                             whatis_more_155_description,

                             whatis_img_more_180kb,
                             whatis_images_alt_less,
                             whatis_missing_alt_attribute,

                             whatis_backlinkscount,
                             whatis_anchortext],
  'ESTIMACIÓN':[text_estimation_404,
                text_estimation_301,

                text_estimation_Remove_Blocking,
                text_Minify_CSS,
                text_Minify_JavaScript,
                text_Properly_Size,
                text_webp,
                text_lazyloading,

                text_falta_h1,
                text_duplicated_h1,
                text_more_70_h1,

                text_titles_less,
                text_duplicate_title,
                text_titles_less_30,
                text_titles_more_60,

                text_description_less,
                text_duplicate_description,
                text_description_more_155,

                text_img_more_180kb,
                text_alt_less,
                text_missing_alt_attribute,

                text_domains,
                text_anchortext],
  'CANTIDAD': [count_404,
               count_301,

               count_Remove_Render_Blocking,
               count_Minify_CSS,
               count_Minify_JavaScript,
               count_Properly_Size_Images,
               count_webp,
               count_lazy,

               count_falta_h1,
               count_duplicated_h1,
               count_more_70_h1,

               count_titles_less,
               count_duplicate_title,
               count_titles_less_30,
               count_titles_more_60,

               count_description_less,
               count_duplicate_description,
               count_description_more_155,

               count_img_more_180kb,
               count_alt_less,
               count_missing_alt_attribute,

               count_domains,
               county_anchortext],
  'PRIORIDAD': ['Media',
                'Media',

                'Media',
                'Media',
                'Media',
                'Media',
                'Media',
                'Media',

                'Alta',
                'Alta',
                'Alta',

                'Alta',
                'Alta',
                'Alta',
                'Alta',

                'Media',
                'Media',
                'Media',

                'Media',
                'Media',
                'Media',

                'Baja',
                'Baja'],
  'ROL' :['Implementador/ frontend',
          'Implementador/ frontend',

          'Frontend',
          'Frontend',
          'Frontend',
          'Frontend',
          'Frontend',
          'Implementador/ frontend',

          'Estratega de contenidos',
          'Estratega de contenidos',
          'Estratega de contenidos',

          'Estratega de contenidos',
          'Estratega de contenidos',
          'Estratega de contenidos',
          'Estratega de contenidos',

          'Estratega de contenidos',
          'Estratega de contenidos',
          'Estratega de contenidos',

          'Implementador',
          'Implementador',
          'Implementador',

          'SEO',
          'SEO']
 }

resumen = pd.DataFrame(Resumen)

Exportación de auditoría

In [ ]:
rute= "/content/drive/MyDrive/Look4SEO/Auditorías/2 Cendiatra/Look4SEO - AUDITORIA-CENDIATRA-"
route_new = rute + str(today) + ".xlsx"
route_new

with pd.ExcelWriter(route_new) as writer:
  df_matriz.to_excel(writer, sheet_name='Matriz de entendimiento')
  resumen.to_excel(writer, sheet_name='Resumen')

  #indexabilidad
  if df_404.empty:
    pass
  else:
    df_404_clear.to_excel(writer, sheet_name='404')

  if df_301.empty:
    pass
  else:
    df_301_clear.to_excel(writer, sheet_name='301')

  if df_Remove_Render_Blocking_Resources.empty:
    pass
  else:
    df_Remove_Render_Blocking_Resources_clear.to_excel(writer, sheet_name='urls bloqueados')

  if df_Minify_CSS.empty:
    pass
  else:
    df_Minify_CSS_clear.to_excel(writer, sheet_name='Minify Css')

  if df_Minify_JavaScript.empty:
    pass
  else:
    df_Minify_JavaScript_clear.to_excel(writer, sheet_name='JS Minify')

  if df_Properly_Size_Images.empty:
    pass
  else:
    df_Properly_Size_Images_clear.to_excel(writer, sheet_name='Properly Size Images')

  if df_lazyloading.empty:
    pass
  else:
    df_lazyloading_clear.to_excel(writer, sheet_name='lazyloading less')
  if df_webp.empty:
    pass
  else:
    df_webp_clear.to_excel(writer, sheet_name='WebP')

  #h1
  if df_falta_h1.empty:
    pass
  else:
    df_falta_h1_clear.to_excel(writer, sheet_name='h1 less')
  if df_duplicated_h1.empty:
    pass
  else:
    df_duplicated_h1_clear.to_excel(writer, sheet_name='Duplicate H1')
  if df_more_70_h1.empty:
    pass
  else:
    df_more_70_h1_clear.to_excel(writer, sheet_name='H1 70+')

  #title
  if df_titles_more_60.empty:
    pass
  else:
    df_titles_more_60_clear.to_excel(writer, sheet_name='Title more 60')
  if df_titles_less_30.empty:
    pass
  else:
    df_titles_less_30_clear.to_excel(writer, sheet_name='Titles less 30')

  #Description
  if df_description_less.empty:
    pass
  else:
    df_description_less_clear.to_excel(writer, sheet_name='less Description')
  if df_duplicate_description.empty:
    pass
  else:
    df_duplicate_description_clear.to_excel(writer, sheet_name='duplicated Description')
  if df_description_more_155.empty:
    pass
  else:
    df_description_more_155_clear.to_excel(writer, sheet_name='Descriptions more 155')

  #Imáges
  if df_img_more_180kb.empty:
    pass
  else:
    df_img_more_180kb_clear.to_excel(writer, sheet_name='img + 180kb')
  if df_alt_less.empty:
    pass
  else:
    df_alt_less_clear.to_excel(writer, sheet_name='img alt less')
  if df_missing_alt_attribute.empty:
    pass
  else:
    df_missing_alt_attribute.to_excel(writer, sheet_name='missing alt attribute')

#Popularidad

if df_domain_count.empty:
    pass
else:
    df_domain_count_clear.to_excel(writer, sheet_name='Backlinks')

if df_anchortext.empty:
    pass
else:
    df_anchortext.to_excel(writer, sheet_name='Anchor text')